In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from random import randint,sample
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:

df = pd.read_excel("/content/drive/MyDrive/NNFL-Assignment2/data5.xlsx").sample(frac = 1)
datan = df.to_numpy()


inp = datan[:,0:datan.shape[1]-1]
op = datan[:,datan.shape[1]-1]


max  =  np.amax(inp,axis=0) ## max  values in each column
input_data  = inp/max
output_data = np.zeros((len(op),3))

#Auto encoding
for i in range(len(op)):
  output_data[i,int(op[i]-1)] = 1

tp1 = int(input_data.shape[0]*0.7)
tp2 = int(input_data.shape[0]*0.8)


input_train_data      = input_data[0:tp1,:]
input_validation_data = input_data[tp1:tp2,:]
input_test_data       = input_data[tp2:input_data.shape[0],:]

output_train_data      = output_data[0:tp1]
output_validation_data = output_data[tp1:tp2]
output_test_data       = output_data[tp2:input_data.shape[0]]

In [5]:

def confusion_matrix(outpt,Y):
  pred = np.argmax(outpt, axis=1)+1
  output_test_data = np.argmax(Y, axis=1)+1
  confusion_matrix = np.zeros((3,3))
  count = 0
  for x in range(len(pred)):
    if(pred[x] == output_test_data[x]):
      count+=1
    if(pred[x] == 1):
      if(output_test_data[x] == 1):
        confusion_matrix[0][0]+=1
      elif(output_test_data[x] == 2):
        confusion_matrix[1][0]+=1
      elif(output_test_data[x] == 3):
        confusion_matrix[2][0]+=1
    elif(pred[x] == 2):
      if(output_test_data[x]   == 1):
        confusion_matrix[0][1]+=1
      elif(output_test_data[x] == 2):
        confusion_matrix[1][1]+=1
      elif(output_test_data[x] == 3):
        confusion_matrix[2][1]+=1
    elif(pred[x] == 3):
      if(output_test_data[x]   == 1):
        confusion_matrix[0][2]+=1
      elif(output_test_data[x] == 2):
        confusion_matrix[1][2]+=1
      elif(output_test_data[x] == 3):
        confusion_matrix[2][2]+=1
  c1 = confusion_matrix[0][0]/np.sum(confusion_matrix[0][:])
  c2 = confusion_matrix[1][1]/np.sum(confusion_matrix[1][:])
  c3 = confusion_matrix[2][2]/np.sum(confusion_matrix[2][:])
  A  = count/len(pred)
  print("class_1  Accuracy  : "+ str( c1))
  print("class_2  Accuracy  : "+ str( c2))
  print("class_3  Accuracy  : "+ str( c3))
  print( "Overall Accuracy  : "+ str( A))
  print("\n")

In [6]:
def perceptron(W,X,B):
  P = (X@W.T) + B
  return 1/(1 + np.exp(-P))



def HNN(alpha,W1,W2,W3,B1,B2,B3,iterations,input_data,output_data):


  fuzz = 6
  WF = np.random.rand(output_data.shape[1],fuzz)
  BF = np.random.rand(output_data.shape[1])
  mu = np.random.rand(input_data.shape[1],fuzz,input_data.shape[1])
  sigma = np.random.rand(input_data.shape[1],fuzz)
  
  
  
  for i in range(iterations):

    layer1 = np.random.rand(input_data.shape[0],input_data.shape[1]*fuzz)
    counter = 0
    for i in range(input_data.shape[1]):
      for j in range(fuzz):
        layer1[:,counter] = np.exp(-np.sum((input_data - mu[i][j])**2,axis=1)/(sigma[i,j]**2))
        counter+=1

    layer2 = np.random.rand(input_data.shape[0],fuzz)
    for i in range(fuzz):
      for j in range(input_data.shape[1]):
        layer2[:,i] *= layer1[:,fuzz*j +i]

    H1_output = perceptron(W1,input_data,B1)
    H2_output = perceptron(W2,H1_output,B2)
    Y_NN      = perceptron(W3,H2_output,B3)

    Y = layer2@WF.T + Y_NN
    error  = -(Y - output_data)
    cost = np.sum(error**2)/(input_data.shape[0]*output_data.shape[1])





    deltaF = np.dot(np.dot(error,WF).T,layer2)
    deltaF1 = np.dot(np.dot(error,WF),layer2.T)

    for i in range(input_data.shape[1]):
      for j in range(fuzz):
        mu[i][j] += 0.001*alpha*np.dot(deltaF1[j],np.sum((input_data - mu[i][j]),axis=1)/(sigma[i,j]**2))


    for i in range(input_data.shape[1]):
      for j in range(fuzz):
        sigma[i][j] += 100*alpha*np.sum(np.dot(deltaF1[j],np.sum((input_data - mu[i][j])**2,axis=1))/(sigma[i,j]**2))
    WF += alpha*np.dot(error.T,layer2) 



    delta3 = error*Y*(1-Y)
    delta2 = np.dot(delta3,W3)*H2_output*(1-H2_output)
    delta1 = np.dot(delta2,W2)*H1_output*(1-H1_output)
    W3 += alpha*np.dot(delta3.T,H2_output)
    W2 += alpha*np.dot(delta2.T,H1_output)
    W1 += alpha*np.dot(delta1.T,input_data)
    B3 += alpha*np.sum(delta3,axis=0)
    B2 += alpha*np.sum(delta2,axis=0)
    B1 += alpha*np.sum(delta1,axis=0)

  layer1 = np.random.rand(input_test_data.shape[0],input_test_data.shape[1]*fuzz)
  counter = 0
  for i in range(input_test_data.shape[1]):
    for j in range(fuzz):
      layer1[:,counter] = np.exp(-np.sum((input_test_data - mu[i][j])**2,axis=1)/(sigma[i,j]**2))
      counter+=1

  layer2 = np.random.rand(input_test_data.shape[0],fuzz)
  for i in range(fuzz):
    for j in range(input_test_data.shape[1]):
      layer2[:,i] *= layer1[:,fuzz*j +i]

  H1_output = perceptron(W1,input_test_data,B1)
  H2_output = perceptron(W2,H1_output,B2)
  Y_NN        = perceptron(W3,H2_output,B3)

  Y = layer2@WF.T + Y_NN
  confusion_matrix(Y,output_test_data)

In [7]:
hidden_layer_size1 = 10
hidden_layer_size2 = 5
W1 = np.random.rand(hidden_layer_size1,input_data.shape[1])
W2 = np.random.rand(hidden_layer_size2,hidden_layer_size1)
W3 = np.random.rand(output_data.shape[1],hidden_layer_size2)
B1 = np.random.rand(hidden_layer_size1)
B2 = np.random.rand(hidden_layer_size2)
B3 = np.random.rand(output_data.shape[1])

HNN(0.1,W1,W2,W3,B1,B2,B3,10000,input_train_data,output_train_data)

class_1  Accuracy  : 0.8421052631578947
class_2  Accuracy  : 1.0
class_3  Accuracy  : 0.6666666666666666
Overall Accuracy  : 0.8571428571428571


